In [2]:
import pandas as pd
import pymysql as pymysql

debug = True

start_date = '2016-01-01'
end_date = '2020-01-01'
resample_interval = '5T'

host = "mariadb.mmto.arizona.edu"
database = "measurements"
table = 'bump'
usr = "webuser"
pw = "Clear7Skies!"

conn = pymysql.connect(host=host, port=3306, user=usr, passwd=pw, db=database)

In [3]:
#  This gets the individual e-series thermocouples parameter names
def get_names(connection):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT ds_name FROM aaa_parameters WHERE ds_name LIKE 'cell_e_series%'"""
            df = pd.read_sql(sql, conn)
    except Exception as e:
        print(f"Error: {e}")
    return df

In [4]:
def get_data(connection, param, database, start_date, end_date):
    df = None
    try:
        # Read data
        with connection.cursor() as cursor:
            sql = f"""SELECT from_unixtime(timestamp/1000) as ts, value as {param} FROM {param} 
                  WHERE timestamp >= UNIX_TIMESTAMP('{start_date}') * 1000
                  AND timestamp < UNIX_TIMESTAMP('{end_date}') * 1000;"""
            # Using a median filter to remove spikes.  Modify as you wish.
            df = pd.read_sql(sql, conn, index_col='ts').resample(resample_interval).median()
    except Exception as e:
        print(f"Error: {e}")
    return df

In [5]:
df_names = get_names(conn)
df_names

,ds_name
0,cell_e_series_backplate_C
1,cell_e_series_backplate_max_C
2,cell_e_series_backplate_min_C
3,cell_e_series_carrier_offset_C
4,cell_e_series_carrier_reference_C
...,...
155,cell_e_series_tc99_C
156,cell_e_series_tc9_C
157,cell_e_series_timestamp
158,cell_e_series_unit_status


In [6]:
drop_list = [
    "cell_e_series_update",
    "cell_e_series_unit_status",
    "cell_e_series_timestamp",
    "cell_e_series_count",
    "cell_e_series_crate_status",
    "cell_e_series_ijb_raw",
    "cell_e_series_interval",
    "cell_e_series_restarted",
    "cell_e_series_server_status"
]
par_list = [
    "cell_e_series_backplate_C",
    "cell_e_series_chamber_ambient_C",
    "cell_e_series_frontplate_C",
    "cell_e_series_in_front_of_primary_C",
    "cell_e_series_lower_plenum_C",
    "cell_e_series_midplate_C",
    "cell_e_series_outside_ambient_C",
    "yankee_temperature",
    "temptrax1_probe2",
    "temptrax1_probe3",
    "temptrax1_probe4",
    "temptrax1_probe6",
    "temptrax3_probe10",
    "temptrax3_probe11",
    "temptrax3_probe12"
]

In [7]:
df_all = None

for name in par_list:
    if debug:
        print(f"Getting {name} values...")
    df = get_data(conn, name, database, start_date, end_date)
    if df_all is None:
        df_all = df
    else:
        if df is not None:
            df_all = df_all.join(df, how='inner')

Getting cell_e_series_backplate_C values...
Getting cell_e_series_chamber_ambient_C values...
Getting cell_e_series_frontplate_C values...
Getting cell_e_series_in_front_of_primary_C values...
Getting cell_e_series_lower_plenum_C values...
Getting cell_e_series_midplate_C values...
Getting cell_e_series_outside_ambient_C values...
Getting yankee_temperature values...
Getting temptrax1_probe2 values...
Getting temptrax1_probe3 values...
Getting temptrax1_probe4 values...
Getting temptrax1_probe6 values...
Getting temptrax3_probe10 values...
Getting temptrax3_probe11 values...
Getting temptrax3_probe12 values...


In [16]:
df_all.tail()

,cell_e_series_backplate_C,cell_e_series_chamber_ambient_C,cell_e_series_frontplate_C,cell_e_series_in_front_of_primary_C,cell_e_series_lower_plenum_C,cell_e_series_midplate_C,cell_e_series_outside_ambient_C,yankee_temperature,temptrax1_probe2,temptrax1_probe3,temptrax1_probe4,temptrax1_probe6,temptrax3_probe10,temptrax3_probe11,temptrax3_probe12
ts,,,,,,,,,,,,,,,
2019-07-26 16:20:00,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
2019-07-26 16:25:00,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
2019-07-26 16:30:00,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
2019-07-26 16:35:00,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9
2019-07-26 16:40:00,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9,-999.9


In [17]:
df_all.shape

(375177, 15)

In [18]:
df_all.to_csv("../halcoll/data/halcoll_temps.csv")

In [ ]:
# temptrax1 2, 3, 4, 6 (ne top, se top, se bottom, hub)
# temptrax3 10, 11, 12 (nw top, sw top, sw bottom)
# yankee_temperature

(.7e-6 ** 0.2) / (0.55e-6 ** 0.2)